In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import os
import sys
import re
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from IPython.display import display, HTML, clear_output, Image
import pandas as pd
from itertools import product
import math
from deepdiff import DeepDiff

In [10]:
# Styling notebook
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML("<style>" + styles + "</style")

css_styling()

In [11]:
sys.path.insert(1, "/workspaces/ou-af/problem2/")

In [12]:
# import de code
import src.oopnclasses as oopnclasses
import src.oopnbuilder as oopnbuilder
import src.diagrambuilder as diagrambuilder
import specs.components as componentspecs
import specs.connections as connectionspecs
import specs.assemblies as assemblyspecs
import specs.tests as testspecs

In [16]:
# gather specs for a system
# system = light + replacedecision + test
components = [componentspecs.light, componentspecs.switch]
connections = [connectionspecs.wire, connectionspecs.wire2]
tests = [testspecs.testObserveHealth]
assembly = {
    "components"  : components,
    "connections" : connections,
    "structure"   : assemblyspecs.structure1,   #light, switch, wire
    "tests"       : tests,
    "testmapping" : testspecs.testmapping4
}

In [22]:
oopn = oopnbuilder.OopnBuilder(assembly)


building system: structure1
number of components: 2


NameError: name 'pd' is not defined

In [26]:
behavior = {
            'PresentPowerInputs' :   ["yes", "no",     "no", "yes"],
            'PresentPowerOutputs':   ["yes", "yes",    "no", "no"],
            'health'             :   ["ok",  "broken", "ok", "broken"]
        }

In [29]:
# determine start / end component names
def addComponentName(name):
    if (re.search(name, startnode)): return startnode
    elif (re.search(name, endnode)): return endnode
    elif (re.search("health", name)): return name + name

def setCptEndComponent():
    dfstates = pd.DataFrame.from_dict(data = behavior)
    dfstates.rename(columns=lambda x: addComponentName(x), inplace=True)
    # transform dataframe back to dict but in different format for comparison
    dfstates.to_dict('index')
    
setCptEndComponent()

TypeError: addComponentName() missing 1 required positional argument: 'name'